In [1]:
import textract

text = textract.process("./文档3.docx")

In [174]:
import re
import pandas as pd
def ItemExtraction(item,text):
    result={}
    text_pre=re.sub('\n','=',text)
    text_use=re.sub('\xc2\xa0','',text_pre) # remove space between item   
    Pattern='.*?'.join([item,'==(.*?)=='])
    matched_string=re.compile(Pattern).findall(text_use)
    n=0
    for value in matched_string:
        
        result.setdefault('matched',{}).update({item+'_'+str(n):value})
        n+=1
    result_pd=pd.DataFrame(result)
    return result_pd

In [131]:
ItemExtraction(item='性别',text=text)

,matched
性别_0,授权代理人：性别：民族：职业：身份证号码：工作单位：联系电话：邮政编码：住址：=委托人因鞍区...
性别_1,男
性别_2,男


In [130]:
def LabResultExtraction(title,ResultItem,text,time='报告日期'):
    result={}
    text_use=re.sub('\n','=',text)
        
    Pattern='.*?'.join([title,time,'(\d\d\d\d-\d\d-\d\d)',ResultItem,'==(.*?)=='])
    matched_string=re.compile(Pattern).findall(text_use)
    for t,value in matched_string:
        
        result.setdefault(ResultItem,{}).update({t:value})
    result_pd=pd.DataFrame(result)
    return result_pd

In [129]:
LabResultExtraction(ResultItem='淋巴细胞计数',text=text,title='检验科临检组检验报告单')

,淋巴细胞计数
2011-03-07,1.5
2011-03-08,0.7
2011-03-12,1.6
2011-03-13,2.1
2011-03-18,0.8
2011-03-20,0.2


In [151]:
def InpatientNote(text,StartKey='病历',EndKey='入院诊断'):
    result={}
    text_pre=re.sub('\n','=',text)
    text_use=re.sub('\xc2\xa0','',text_pre) # remove space between item   
    Pattern=StartKey+'='+'(.*?)'+EndKey
    matched_string=re.compile(Pattern).findall(text_use)
    n=0
    for value in matched_string:
        
        result.setdefault('matched',{}).update({StartKey+'_'+str(n):value})
        n+=1
    result_pd=pd.DataFrame(result)
    return result_pd

In [152]:
InpatientNote(text)

,matched
病历_0,=====主 诉：生长缓慢2年余，反复头痛1月，发现颅内占位病变10天。 ==现病史：家长反...


In [167]:
def ClinicalNote(text,TimeKey='(\d\d\d\d-\d\d-\d\d \d\d:\d\d)',EndKey='医师:'):
    result={}
    text_use=re.sub('\n','=',text)
    Pattern=TimeKey+'=='+'(.*?)'+'='+'(.*?)'+EndKey
    matched_string=re.compile(Pattern).findall(text_use)
    for time,notetype,note in matched_string:
        
        result.setdefault('NoteTime',[]).append(time)
        result.setdefault('NoteType',[]).append(notetype)
        result.setdefault('Note',[]).append(note)
    result_pd=pd.DataFrame(result)
    return result_pd

In [172]:
clinical_note=ClinicalNote(text)

In [173]:
clinical_note.loc[:,['NoteTime','NoteType']]

,NoteTime,NoteType
0,2011-03-06 10:47,病 案 号：
1,2011-03-07 09:00,副教授代主治医师查房
2,2011-03-08 10:00,教授代主治医师查房
3,2011-03-11 10:00,病程记录
4,2011-03-14 09:30,副教授代主治医师查房
5,2011-03-16 08:43,术前小结
6,2011-03-17 15:30,术后首次病程
7,2011-03-17 19:00,转科小结
8,2011-03-17 19:23,转入记录
9,2011-03-18 09:50,教授代主治医师查房
